# Health Calculators

This cell contains the code from `health_calculators.py`. It defines classes for calculating BMI, BMR, and TDEE.

In [ ]:
from datetime import datetime


class BMI:
    @staticmethod
    def calculate(weight: float, height: float) -> float:
        return round(weight / ((height / 100) ** 2), 2) if height > 0 else 0.0


class BMR:
    @staticmethod
    def calculate(weight: float, height: float, bd: str, sex: str) -> float:
        birth_year = int(bd.split(":")[2])
        age = datetime.now().year - birth_year
        sex = sex.lower()
        if sex == "male":
            bmr = 10 * weight + 6.25 * height - 5 * age + 5
        elif sex == "female":
            bmr = 10 * weight + 6.25 * height - 5 * age - 161
        elif sex == "other":
            bmr = 10 * weight + 6.25 * height - 5 * age - 78
        else:
            raise ValueError("Sex must be 'male' or 'female'")
        return round(bmr, 2)


class TDEE:
    activity_factors = {
        "sedentary": 1.2,
        "light": 1.375,
        "moderate": 1.55,
        "active": 1.725,
        "very_active": 1.9,
    }

    @classmethod
    def calculate(cls, bmr: float, activity_level: str) -> float:
        if activity_level not in cls.activity_factors:
            raise ValueError(
                "Activity level must be one of: 'sedentary', 'light', 'moderate', 'active', 'very_active'"
            )
        return round(bmr * cls.activity_factors[activity_level], 2)

# Personal Information Manager

This cell contains the code from `personal_manager.py`. It manages a user's personal data and calculates health metrics using the classes defined above.

In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime

class PersonalManager:
    def __init__(self, data_folder="data"):
        self.data_folder = Path(data_folder)
        self.data_folder.mkdir(exist_ok=True)
        self.file = self.data_folder / "personal_info.csv"

    def load_last_entry(self, user="default") -> dict:
        if self.file.exists() and self.file.stat().st_size > 0:
            df = pd.read_csv(self.file)
            user_df = df[df["name"] == user] if "name" in df.columns else df
            if not user_df.empty:
                record = user_df.iloc[-1].to_dict()

                # Convert height/weight to preferred unit for display
                height_unit = record.get("height_unit", "cm")
                weight_unit = record.get("weight_unit", "kg")
                height = record.get("height", 0)
                weight = record.get("weight", 0)

                if height_unit == "ft":
                    record["height"] = round(height / 30.48, 2)
                if weight_unit == "lbs":
                    record["weight"] = round(weight * 2.20462, 2)

                record["height_unit"] = height_unit
                record["weight_unit"] = weight_unit
                return record
        return {}

    def save_info(
        self,
        user: str,
        sex: str,
        bd: str,
        height: float,
        weight: float,
        activity_level: str,
        height_unit="cm",
        weight_unit="kg",
    ) -> tuple:
        # Convert input to cm/kg for storage
        if height_unit == "ft":
            height = float(height) * 30.48
        else:
            height = float(height)

        if weight_unit == "lbs":
            weight = float(weight) / 2.20462
        else:
            weight = float(weight)

        bmi = BMI.calculate(weight, height)
        bmr = BMR.calculate(weight, height, bd, sex) if height > 0 else 0
        tdee = TDEE.calculate(bmr, activity_level) if bmr > 0 else 0

        record = pd.DataFrame(
            [
                {
                    "time": datetime.now(),
                    "name": user,
                    "sex": sex,
                    "bd": bd,
                    "height": height,
                    "weight": weight,
                    "bmi": bmi,
                    "bmr": bmr,
                    "tdee": tdee,
                    "activity_level": activity_level,
                    "height_unit": height_unit,
                    "weight_unit": weight_unit,
                }
            ]
        )

        if self.file.exists() and self.file.stat().st_size > 0:
            record.to_csv(self.file, mode="a", header=False, index=False)
        else:
            record.to_csv(self.file, index=False)

        return bmi, bmr, tdee

# Food Manager

This cell contains the code from `food_manager.py`. It handles logging food intake and retrieving food data.

In [ ]:
import pandas as pd
from pathlib import Path
from datetime import datetime


class FoodManager:
    def __init__(self, data_folder="data"):
        self.data_folder = Path(data_folder)
        self.data_folder.mkdir(exist_ok=True)
        self.food_file = self.data_folder / "food_data.csv"
        self.cal_file = self.data_folder / "cal_rec.csv"
        if not self.food_file.exists():
            pd.DataFrame(columns=["food", "cal"]).to_csv(self.food_file, index=False)

    def get_food_list(self) -> list:
        df = pd.read_csv(self.food_file)
        return df["food"].tolist() if not df.empty else []

    def add_food(self, user: str, food_name: str) -> tuple:
        df = pd.read_csv(self.food_file)
        row = df[df["food"] == food_name]
        if row.empty:
            return False, f"❌ Food '{food_name}' not found!"
        cal = int(row.iloc[0]["cal"])
        record = pd.DataFrame(
            [{"time": datetime.now(), "name": user, "food": food_name, "cal": cal}]
        )
        if self.cal_file.exists() and self.cal_file.stat().st_size > 0:
            record.to_csv(self.cal_file, mode="a", header=False, index=False)
        else:
            record.to_csv(self.cal_file, index=False)
        return True, f"✅ Added {food_name} ({cal} cal)!"

# Chart Manager

This cell contains the code from `chart_manager.py`. It's responsible for creating visualizations of the health data.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from pathlib import Path


class ChartManager:
    def __init__(
        self, cal_file="data/cal_rec.csv", personal_file="data/personal_info.csv"
    ):
        self.cal_file = cal_file
        self.personal_file = personal_file

    def build_last_7_days_chart(self, user="default"):
        # Load CSVs
        cal_df = (
            pd.read_csv(self.cal_file, parse_dates=["time"])
            if Path(self.cal_file).exists()
            else pd.DataFrame()
        )
        personal_df = (
            pd.read_csv(self.personal_file, parse_dates=["time"])
            if Path(self.personal_file).exists()
            else pd.DataFrame()
        )

        today = datetime.today().date()
        date_range = pd.date_range(end=today, periods=7).date

        # Filter by user
        cal_df = (
            cal_df[cal_df["name"] == user]
            if not cal_df.empty
            else pd.DataFrame(columns=["time", "food", "cal"])
        )
        personal_df = (
            personal_df[personal_df["name"] == user]
            if not personal_df.empty
            else pd.DataFrame(columns=["time", "tdee"])
        )

        # Process food intake
        if not cal_df.empty:
            cal_df["date"] = cal_df["time"].dt.date
            # Sum calories
            food_summary = cal_df.groupby(["date", "food"])["cal"].sum().reset_index()
            food_pivot = food_summary.pivot(
                index="date", columns="food", values="cal"
            ).fillna(0)

            # Count occurrences
            food_count = (
                cal_df.groupby(["date", "food"]).size().reset_index(name="count")
            )
            count_pivot = food_count.pivot(
                index="date", columns="food", values="count"
            ).fillna(0)
        else:
            food_pivot = pd.DataFrame(index=date_range)
            count_pivot = pd.DataFrame(index=date_range)

        food_pivot = food_pivot.reindex(date_range, fill_value=0)
        count_pivot = count_pivot.reindex(date_range, fill_value=0)

        total_intake = food_pivot.sum(axis=1)

        # Process TDEE
        if not personal_df.empty:
            personal_df["date"] = personal_df["time"].dt.date
            tdee_series = personal_df.groupby("date")["tdee"].last()
            tdee_series = tdee_series.reindex(date_range).ffill().fillna(0)
        else:
            tdee_series = pd.Series([0] * 7, index=date_range)

        # Build figure
        fig = go.Figure()
        colors = px.colors.qualitative.Plotly

        # Stacked bar for food
        for i, food_name in enumerate(food_pivot.columns):
            fig.add_trace(
                go.Bar(
                    x=date_range,
                    y=food_pivot[food_name],
                    name=food_name,
                    marker_color=colors[i % len(colors)],
                    hovertemplate=[
                        f"{food_pivot.loc[d, food_name]} kcal of {food_name} ({int(count_pivot.loc[d, food_name])} times)<extra></extra>"
                        for d in date_range
                    ],
                )
            )

        tdee_colors = [
            "green" if tdee_series[d] >= total_intake[d] else "red" for d in date_range
        ]

        # Line for TDEE
        fig.add_trace(
            go.Scatter(
                x=date_range,
                y=tdee_series,
                mode="lines+markers",
                name="TDEE",
                line=dict(color="gray", width=2),
                marker=dict(size=10, color=tdee_colors, symbol="circle"),
                hovertemplate="TDEE: %{y} kcal<extra></extra>",
            )
        )

        fig.update_layout(
            barmode="stack",
            title=f"Last 7 Days: TDEE vs Food Intake ({user})",
            xaxis_title="Date",
            yaxis_title="Calories",
            legend_title="Foods / TDEE",
            template="plotly_white",
            xaxis=dict(tickformat="%Y-%m-%d"),
        )

        return fig

# Gradio User Interface

This cell contains the complete User Interface code from `ui.py`. It defines the Gradio theme and the `AppUI` class which builds the interactive web application.

In [ ]:
from __future__ import annotations
import gradio as gr
from datetime import datetime
import plotly.graph_objects as go
from typing import Iterable
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes


class HealthCalcTheme(Base):
    """
    Healthcare-friendly theme using exact hex colors.
    Uses only the safe Seafoam snippet attributes.
    """

    def __init__(
        self,
        *,
        primary_hue: str = "green",  # placeholder for Base; hex used in set()
        secondary_hue: str = "teal",  # placeholder
        neutral_hue: str = "stone",  # placeholder
        spacing_size: sizes.Size | str = sizes.spacing_md,
        radius_size: sizes.Size | str = sizes.radius_md,
        text_size: sizes.Size | str = sizes.text_md,
        font: fonts.Font | str | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Quicksand"),
            "ui-sans-serif",
            "sans-serif",
        ),
        font_mono: fonts.Font | str | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("IBM Plex Mono"),
            "ui-monospace",
            "monospace",
        ),
    ):
        super().__init__(
            primary_hue=primary_hue,
            secondary_hue=secondary_hue,
            neutral_hue=neutral_hue,
            spacing_size=spacing_size,
            radius_size=radius_size,
            text_size=text_size,
            font=font,
            font_mono=font_mono,
        )

        # Exact hex colors for your palette
        HEX_PRIMARY = "#3e6b3e"  # dark green
        HEX_SECONDARY = "#6cb49b"  # teal-green
        HEX_ACCENT = "#d9a441"  # gold
        HEX_NEUTRAL = "#8c5e3c"  # brown

        # Only safe Seafoam snippet attributes
        super().set(
            body_background_fill=f"linear-gradient(135deg, {HEX_PRIMARY}20, {HEX_SECONDARY}20)",  # subtle gradient
            body_background_fill_dark=f"linear-gradient(135deg, {HEX_PRIMARY}90, {HEX_SECONDARY}80)",
            button_primary_background_fill=f"linear-gradient(90deg, {HEX_PRIMARY}, {HEX_SECONDARY})",
            button_primary_background_fill_hover=f"linear-gradient(90deg, {HEX_ACCENT}, {HEX_SECONDARY})",
            button_primary_text_color="white",
            button_primary_background_fill_dark=f"linear-gradient(90deg, {HEX_PRIMARY}, {HEX_NEUTRAL})",
            slider_color=f"{HEX_SECONDARY}",
            slider_color_dark=f"{HEX_ACCENT}",
            block_title_text_weight="600",
            block_border_width="3px",
            block_shadow="*shadow_drop_lg",
            button_primary_shadow="*shadow_drop_lg",
            button_large_padding="32px",
        )


HCT = HealthCalcTheme()


class AppUI:
    def __init__(self, personal_manager, food_manager, chart_manager):
        self.personal_manager = personal_manager
        self.food_manager = food_manager
        self.chart_manager = chart_manager
        self.session_name = None

    @staticmethod
    def _empty_chart():
        fig = go.Figure()
        fig.update_layout(
            title="No data yet",
            xaxis_title="Date",
            yaxis_title="Calories",
            template="plotly_white",
        )
        return fig

    @staticmethod
    def _announcement(msg, success=True):
        # Hex colors from HealthCalcTheme
        HEX_SUCCESS_BG = "#6cb49b"  # teal-green background for success
        HEX_SUCCESS_TEXT = "#FFFFFF"
        HEX_ERROR_BG = "#d9a441"  # gold-ish background for error/warning
        HEX_ERROR_TEXT = "#b33a3a"  # red text for error/warning

        color_bg = HEX_SUCCESS_BG if success else HEX_ERROR_BG
        color_text = HEX_SUCCESS_TEXT if success else HEX_ERROR_TEXT

        html_content = f"""
        <div id='popup-banner' style='
            position: fixed;
            top: 20px;
            right: 20px;
            background:{color_bg};
            color:{color_text};
            padding:15px 25px;
            border-radius:8px;
            box-shadow: 0px 4px 8px rgba(0,0,0,0.2);
            z-index:9999;
            font-weight:bold;
        '>{msg}</div>
        """
        return gr.update(value=html_content, visible=True)

    # --- LOGIN ---
    def login_handler(self, login_name):
        self.session_name = login_name
        name_box_val = login_name
        record = self.personal_manager.load_last_entry(login_name)

        if record:
            chart = self.chart_manager.build_last_7_days_chart(login_name)
            bmi = record.get("bmi", 0)
            bmr = record.get("bmr", 0)
            tdee = record.get("tdee", 0)
            bd_parts = record.get("bd", "1:Jan:2000").split(":")
            sex_val = record.get("sex", "Male")
            height_val = record.get("height", 0)
            weight_val = record.get("weight", 0)
            height_unit = record.get("height_unit", "cm")
            weight_unit = record.get("weight_unit", "kg")
            activity_val = record.get("activity_level", "sedentary")
            reverse_map = {
                "sedentary": "ออกกำลังกายน้อยมาก หรือไม่ออกเลย",
                "light": "ออกกำลังกาย 1-3 ครั้งต่อสัปดาห์",
                "moderate": "ออกกำลังกาย 4-5 ครั้งต่อสัปดาห์",
                "active": "ออกกำลังกาย 6-7 ครั้งต่อสัปดาห์",
                "very_active": "ออกกำลังกายวันละ 2 ครั้งขึ้นไป",
            }
            activity_val = reverse_map.get(activity_val, "ออกกำลังกายน้อยมาก หรือไม่ออกเลย")
            return (
                gr.update(visible=False),
                name_box_val,
                gr.update(visible=True),
                sex_val,
                bd_parts[0],
                bd_parts[1],
                bd_parts[2],
                height_val,
                weight_val,
                height_unit,
                weight_unit,
                activity_val,
                chart,
                bmi,
                bmr,
                tdee,
                gr.update(value="", visible=False),
                gr.update(value="", visible=False),
            )
        else:
            return (
                gr.update(visible=False),
                name_box_val,
                gr.update(visible=True),
                "Male",
                "1",
                "Jan",
                str(datetime.now().year - 25),
                0,
                0,
                "cm",
                "kg",
                "ออกกำลังกายน้อยมาก หรือไม่ออกเลย",
                self._empty_chart(),
                0,
                0,
                0,
                gr.update(value="", visible=False),
                gr.update(value="", visible=False),
            )

    # --- SAVE INFO ---
    def save_info_handler(
        self,
        name,
        sex,
        bd_day,
        bd_month,
        bd_year,
        height,
        weight,
        height_unit,
        weight_unit,
        activity,
    ):
        bd = f"{bd_day}:{bd_month}:{bd_year}"
        activity_map = {
            "ออกกำลังกายน้อยมาก หรือไม่ออกเลย": "sedentary",
            "ออกกำลังกาย 1-3 ครั้งต่อสัปดาห์": "light",
            "ออกกำลังกาย 4-5 ครั้งต่อสัปดาห์": "moderate",
            "ออกกำลังกาย 6-7 ครั้งต่อสัปดาห์": "active",
            "ออกกำลังกายวันละ 2 ครั้งขึ้นไป": "very_active",
        }
        activity_key = activity_map.get(activity, "sedentary")
        bmi, bmr, tdee = self.personal_manager.save_info(
            name, sex, bd, height, weight, activity_key, height_unit, weight_unit
        )
        chart = self.chart_manager.build_last_7_days_chart(name)
        return (
            self._announcement(f"✅ Saved! BMI:{bmi}, BMR:{bmr}, TDEE:{tdee}"),
            chart,
            bmi,
            bmr,
            tdee,
        )

    # --- ADD FOOD ---
    def add_food_handler(self, name, food_name, quantity):
        msg = ""
        success = True
        for _ in range(int(quantity)):
            s, m = self.food_manager.add_food(name, food_name)
            msg += m + "<br>"
            if not s:
                success = False
        record = self.personal_manager.load_last_entry(name)
        chart = self.chart_manager.build_last_7_days_chart(name)
        bmi = record.get("bmi", 0)
        bmr = record.get("bmr", 0)
        tdee = record.get("tdee", 0)
        return (
            AppUI._announcement(msg, success),
            chart,
            bmi,
            bmr,
            tdee,
        )

    # --- LOGOUT ---
    def logout_handler(self):
        self.session_name = None
        return gr.update(visible=True), gr.update(visible=False), gr.update(value="")

    # --- UI ---
    def build_ui(self):
        with gr.Blocks(theme=HCT) as demo:
            # LOGIN PAGE
            with gr.Group(visible=True) as login_page:
                login_name = gr.Textbox(label="Enter your name")
                login_btn = gr.Button("Login", variant="primary")

            # APP PAGE
            with gr.Group(visible=False) as app_page:
                with gr.Tabs() as tabs:
                    # MAIN TAB
                    with gr.Tab("Main"):
                        name_box = gr.Textbox(label="Full Name", interactive=False)
                        chart_plot = gr.Plot(self._empty_chart())
                        with gr.Row():
                            bmi_out = gr.Number(label="BMI", interactive=False)
                            bmr_out = gr.Number(label="BMR", interactive=False)
                            tdee_out = gr.Number(label="TDEE", interactive=False)

                        # --- Food selection ---
                        with gr.Row():
                            food_dropdown = gr.Dropdown(
                                choices=self.food_manager.get_food_list(),
                                label="Select Food",
                                scale=3,
                            )
                            food_quantity = gr.Slider(
                                minimum=1,
                                maximum=10,
                                value=1,
                                step=1,
                                label="Quantity (times to add)",
                                scale=2,
                            )
                        add_btn = gr.Button("Add Food", variant="primary")
                        popup_food = gr.HTML(value="", visible=False)

                        add_btn.click(
                            fn=self.add_food_handler,
                            inputs=[name_box, food_dropdown, food_quantity],
                            outputs=[
                                popup_food,
                                chart_plot,
                                bmi_out,
                                bmr_out,
                                tdee_out,
                            ],
                        )
                        popup_food.change(
                            fn=lambda x: gr.update(visible=False),
                            inputs=[popup_food],
                            outputs=[popup_food],
                            queue=False,
                        )

                    # PERSONAL INFO TAB
                    with gr.Tab("Personal Info"):
                        sex = gr.Dropdown(["Male", "Female", "Other"], label="Sex")
                        days = [str(i) for i in range(1, 32)]
                        months = [
                            "Jan",
                            "Feb",
                            "Mar",
                            "Apr",
                            "May",
                            "Jun",
                            "Jul",
                            "Aug",
                            "Sep",
                            "Oct",
                            "Nov",
                            "Dec",
                        ]
                        years = [
                            str(y)
                            for y in range(
                                datetime.now().year - 100, datetime.now().year + 1
                            )
                        ]
                        bd_day = gr.Dropdown(days, label="Day")
                        bd_month = gr.Dropdown(months, label="Month")
                        bd_year = gr.Dropdown(years, label="Year")

                        # Height + Unit
                        with gr.Row():
                            height = gr.Number(label="Height", scale=2)
                            height_unit = gr.Dropdown(
                                choices=["cm", "ft"], label="Unit", value="cm", scale=0
                            )

                        # Weight + Unit
                        with gr.Row():
                            weight = gr.Number(label="Weight", scale=2)
                            weight_unit = gr.Dropdown(
                                choices=["kg", "lbs"], label="Unit", value="kg", scale=0
                            )

                        activity = gr.Dropdown(
                            choices=[
                                "ออกกำลังกายน้อยมาก หรือไม่ออกเลย",
                                "ออกกำลังกาย 1-3 ครั้งต่อสัปดาห์",
                                "ออกกำลังกาย 4-5 ครั้งต่อสัปดาห์",
                                "ออกกำลังกาย 6-7 ครั้งต่อสัปดาห์",
                                "ออกกำลังกายวันละ 2 ครั้งขึ้นไป",
                            ],
                            label="Activity Level",
                        )
                        save_btn = gr.Button("💾 Save Info", variant="primary")
                        popup_info = gr.HTML(value="", visible=False)

                        save_btn.click(
                            fn=self.save_info_handler,
                            inputs=[
                                name_box,
                                sex,
                                bd_day,
                                bd_month,
                                bd_year,
                                height,
                                weight,
                                height_unit,
                                weight_unit,
                                activity,
                            ],
                            outputs=[
                                popup_info,
                                chart_plot,
                                bmi_out,
                                bmr_out,
                                tdee_out,
                            ],
                        )

                        logout_btn = gr.Button("🔄 Logout", variant="secondary")
                        logout_btn.click(
                            fn=self.logout_handler,
                            outputs=[login_page, app_page, login_name],
                        )

            login_btn.click(
                fn=self.login_handler,
                inputs=[login_name],
                outputs=[
                    login_page,
                    name_box,
                    app_page,
                    sex,
                    bd_day,
                    bd_month,
                    bd_year,
                    height,
                    weight,
                    height_unit,
                    weight_unit,
                    activity,
                    chart_plot,
                    bmi_out,
                    bmr_out,
                    tdee_out,
                    gr.HTML(value="", visible=False),  # popup_food
                    gr.HTML(value="", visible=False),  # popup_info
                ],
            )
        return demo

# Main Application Logic

This cell initializes all the manager classes and the UI, then launches the Gradio application.

In [ ]:
# Install necessary libraries for the notebook environment
%pip install gradio pandas plotly

# Initialize managers (classes are defined in the cells above)
personal_manager = PersonalManager()
food_manager = FoodManager()
chart_manager = ChartManager()

# Initialize UI (AppUI class is defined in the cell above)
app_ui = AppUI(personal_manager, food_manager, chart_manager)

# Build and launch the Gradio interface
demo = app_ui.build_ui()
demo.launch()